In [161]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

In [162]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch.cuda.empty_cache()


In [163]:
from datasets import Dataset
data = pd.read_csv("data/train/train.En.csv",index_col=0)
data.dropna(subset=["tweet","sarcastic"],axis=0)

df = data[["tweet","sarcastic"]].rename(columns={"tweet":"text","sarcastic":"label"}).dropna().reset_index(drop=True)
df_test = pd.read_csv("data/test/task_A_En_test.csv").rename(columns={"sarcastic":"label"}).dropna()

ds = Dataset.from_pandas(pd.concat([df,df_test]).reset_index(drop=True))
ds = ds.train_test_split(test_size=0.2)

In [164]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [165]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [166]:
# tokenized_train = train_ds.map(preprocess_function,batched=True)
# tokenized_test = test_ds.map(preprocess_function,batched=True)
tokenized = ds.map(preprocess_function,batched=True)




Map: 100%|██████████| 3893/3893 [00:00<00:00, 9838.27 examples/s]

Map: 100%|██████████| 974/974 [00:00<00:00, 16051.88 examples/s]


In [167]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [168]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [169]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: 0, 1: 1}

label2id = {0: 0, 1: 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [170]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 1.95 GiB total capacity; 1.28 GiB already allocated; 153.88 MiB free; 1.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
trainer.train()


  0%|          | 0/488 [03:49<?, ?it/s]
/home/mhd/dev/sarcasm_detector/.venv/lib64/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 1.95 GiB total capacity; 1.28 GiB already allocated; 153.88 MiB free; 1.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF